# ABS Quarterly Business Indicators 5676

## Python set-up

In [1]:
# analytic imports
import textwrap
import pandas as pd
import readabs as ra
from readabs import metacol as mc

In [2]:
# local imports
from abs_helper import get_abs_data
from abs_plotting import fix_abs_title
from henderson import hma
from mgplot import (
    multi_start,
    line_plot_finalise,
    series_growth_plot_finalise,
)

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# display charts
SHOW = False

## Get data from ABS

In [4]:
abs_dict, meta, source, _ = get_abs_data("5676.0")
plot_times = 0, -29

In [5]:
# A quick look at table names
textwrap.wrap(", ".join(abs_dict.keys()), width=80)

['5676001, 5676002, 5676003, 5676004, 5676005, 5676006, 5676007, 5676008, 5676009,',
 '56760010, 56760011, 56760012, 56760013, 56760014, 56760015, 56760016, 56760017,',
 '56760018, 56760019, 56760020, 56760021, 56760022, 56760023, 56760024, 56760025']

In [6]:
print("Latest data: ", abs_dict["5676001"].index[-1])

Latest data:  2025Q1


## Plot

### Headline charts

In [7]:
def headline():
    """Produce headline charts."""

    series_type = "Seasonally Adjusted"
    headlines = [
        "Inventories ;  Total (State) ;  Total (Industry) ;  Chain Volume Measures ;",
        "Profit before Income Tax ;  Total (State) ;  Total (Industry) ;  Current Price ;  CORP ;",
        "Gross Operating Profits ;  Total (State) ;  Total (Industry) ;  "
        "Current Price ;  TOTAL (SCP_SCOPE) ;",
        "Wages ;  Total (State) ;  Total (Industry) ;  Current Price ;",
    ]
    recent = plot_times[1]
    for h in headlines:
        # get the data
        row = meta[
            (meta[mc.stype] == series_type) & meta[mc.did].str.contains(h, regex=False)
        ].iloc[0]
        series_id, units, table = row[mc.id], row[mc.unit], row[mc.table]
        series, units = ra.recalibrate(abs_dict[table][series_id], units)

        # plot
        lfooter = f"Australia. {series_type.capitalize()}. "
        title, lfooter = fix_abs_title(f"Business indicators: {row[mc.did]}", lfooter)
        multi_start(
            series,
            function=line_plot_finalise,
            starts=plot_times,
            title=title,
            ylabel=units,
            pre_tag="!",
            rfooter=source,
            lfooter=lfooter,
            show=SHOW,
        )

        series_growth_plot_finalise(
            series,
            plot_from=-19,
            title=f"Growth in {title.title()}",
            pre_tag="!",
            rfooter=source,
            lfooter=lfooter,
            show=SHOW,
        )


headline()

### Profits vs wages

In [8]:
def profits_v_wages():
    """Produce provists versus wages charts."""

    series_type = "Seasonally Adjusted"

    wages_table = "56760017"
    wages_did = "Wages ;  Total (State) ;  Total (Industry) ;  Current Price ;"
    wages_name = wages_did.split(";", maxsplit=1)[0].strip()
    _table, wages_id, wage_units = ra.find_abs_id(
        meta,
        {
            wages_table: mc.table,
            series_type: mc.stype,
            wages_did: mc.did,
        },
        verbose=False,
    )
    wages = abs_dict[wages_table][wages_id].dropna()

    profits_table = "56760015"
    profits_did = (
        "Gross Operating Profits ;  Total (State) ;  Total (Industry) ;  "
        "Current Price ;  TOTAL (SCP_SCOPE) ;"
    )
    profits_name = profits_did.split(";", maxsplit=1)[0].strip()
    _table, profits_id, profits_units = ra.find_abs_id(
        meta,
        {
            profits_table: mc.table,
            series_type: mc.stype,
            profits_did: mc.did,
        },
        verbose=False,
    )
    profits = abs_dict[profits_table][profits_id].dropna()

    # plot profits as a share of wages and prices
    assert wage_units == profits_units
    profit_wage_share = profits / (profits + wages) * 100
    hma_term = 7
    trend = hma(profit_wage_share.dropna(), hma_term)
    df = pd.DataFrame(
        {
            f"{profits_name} share": profit_wage_share,
            "Henderson moving average": trend,
        }
    )

    line_plot_finalise(
        df,
        title="Profits as a share of profits plus wages",
        ylabel="Per cent",
        rfooter=source,
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{hma_term}-term Henderson moving average. ",
        show=SHOW,
    )

    # Plot values
    assert "Current Price" in wages_did and "Current Price" in profits_did
    assert "Millions" in wage_units
    df = pd.DataFrame({"Wages": (wages / 1000), "Profits": (profits / 1000)})
    line_plot_finalise(
        df,
        title="Profits vs Wages",
        ylabel="$ Billions",
        rfooter=source,
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        show=SHOW,
    )

    # plot wage and profits index (note starting point ptoblem)
    assert profits.index[0] == wages.index[0]  # common start
    profits_index = profits / profits.iloc[0] * 100
    wages_index = wages / wages.iloc[0] * 100
    df = pd.DataFrame(
        {
            f"{wages_name} index": wages_index,
            f"{profits_name} index": profits_index,
        }
    )
    line_plot_finalise(
        df,
        title="Profits index vs Wages index",
        ylabel="Index",
        rfooter=source,
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        show=SHOW,
    )


profits_v_wages()

### Inventories by industry sector

In [9]:
def inventories():
    """Produce Inventory charts."""

    # identify the plotable items ...
    series_type = "Seasonally Adjusted"
    measure_type = "Chain Volume Measures"
    rows = meta[
        (meta[mc.stype] == series_type)
        & meta[mc.did].str.contains("Inventories", regex=False)
        & meta[mc.did].str.contains(measure_type, regex=False)
        & meta[mc.unit].str.contains("$", regex=False)
    ]

    for _, row in rows.iterrows():
        # get the data for wacg poltable item
        series_id, units, table = row[mc.id], row[mc.unit], row[mc.table]
        data = abs_dict[table]
        series, units = ra.recalibrate(data[series_id], units)

        # plot the data
        lfooter = f"Australia. {series_type.capitalize()}. "
        title, lfooter = fix_abs_title(row[mc.did], lfooter)
        multi_start(
            series,
            function=line_plot_finalise,
            starts=[0,-17],
            title=title,
            ylabel=units,
            rfooter=source,
            lfooter=lfooter,
            show=SHOW,
        )


inventories()

## Wages Growth

In [10]:
def wage_growth():
    """Produce wage growth charts."""

    series_type = "Seasonally Adjusted"
    series_did = "Wages"
    rows = meta[
        meta[mc.did].str.contains(series_did, regex=False)
        & (meta[mc.stype] == series_type)
        & meta[mc.unit].str.contains("$", regex=False)
    ]

    for _, row in rows.iterrows():
        # get the data
        series_id, units, table = row[mc.id], row[mc.unit], row[mc.table]
        data = abs_dict[table]
        series, units = ra.recalibrate(data[series_id], units)
        lfooter = f"Australia. {series_type.capitalize()}. "
        title, lfooter = fix_abs_title(row[mc.did], lfooter)
        series_growth_plot_finalise(
            series,
            plot_from=-19,
            title=f"Growth: {title}",
            rfooter=f"{source} T{table}",
            lfooter=lfooter,
            show=SHOW,
        )


wage_growth()

## Finished

In [11]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-06-12 09:24:36

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.3.0

conda environment: n/a

Compiler    : Clang 20.1.0 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas : 2.3.0
mgplot : 0.1.11
readabs: 0.0.31

Watermark: 2.5.0



In [12]:
print("Finished")

Finished
